In [11]:
### @author     Peitsa Rautio
### @version    24.09.2024

# Summary:

# This .ipynb program imports a saved JSON pandapipes model, runs a
# time series simulation and writes time series simulation data to disk.

# Pandapipes and Pandapower © Copyright 2020-2024 by Fraunhofer Institute for
# Energy Economics and Energy System Technology (IEE), Kassel,
# and University of Kassel.

In [12]:
import holoviews as hv
import hvplot.pandas
import numpy as np
import pandas as pd
import pandapipes as pp
import panel as pn
import pandapower.control as control
from pandapower.timeseries import DFData
from pandapower.timeseries import OutputWriter
from pandapipes.timeseries import run_timeseries
from bokeh.palettes import RdBu

hv.extension('bokeh')
pn.extension()

In [13]:
## Simulation options

# Load model from JSON

net = pp.from_json("output/models/RHN-Full.json")

# Load time series data (Excel)
ts_profiles_source = pd.read_excel("data/timeseries_source.xlsx")
ts_profiles_sink = pd.read_excel("data/timeseries_sink.xlsx")

# Load time series data (csv)
# ts_profiles_source = pd.read_csv("data/filenamehere")
# ts_profiles_sink = pd.read_csv("data/filenamehere")

# Create data source
ds_source = DFData(ts_profiles_source)
ds_sink = DFData(ts_profiles_sink)

# Define number of simulation time steps (15 min each) (use len(ts_profiles_source) to set to length of timeseries source file)
n_ts = len(ts_profiles_source)

In [14]:
## Create element controllers

const_source = control.ConstControl(net, element='circ_pump_pressure', variable='mdot_flow_kg_per_s',
                                    element_index=net.circ_pump_pressure.index.values,
                                    data_source=ds_source,
                                    profile_name=net.circ_pump_pressure.index.values,
                                    recycle=True,transient=True)

const_sink = control.ConstControl(net, element='heat_consumer', variable='controlled_mdot_kg_per_s',
                                  element_index=net.heat_consumer.index.values,
                                  data_source=ds_sink,
                                  profile_name=net.heat_consumer.index.values,
                                  recycle=True,transient=True)

In [15]:
## Create output writer and run time series

log_variables = [('res_junction', 'p_bar'), ('res_pipe', 'v_mean_m_per_s'),
                 ('res_pipe', 'reynolds'), ('res_pipe', 'lambda'),
                 ('res_heat_consumer', 'mdot_from_kg_per_s'),
                 ('res_heat_consumer', 't_from_k'),
                 ('res_heat_consumer', 't_to_k'),
                 ('res_circ_pump_pressure', 'mdot_flow_kg_per_s'),
                 ('res_ext_grid', 'mdot_kg_per_s')]

# ow = OutputWriter(net, n_ts, output_path="./output/timeseries/", output_file_type=".xlsx", log_variables=log_variables)
ow = OutputWriter(net, n_ts, output_path="./output/timeseries/", output_file_type=".csv", log_variables=log_variables)

run_timeseries(net,n_ts,transient=True)

# Print resultant node and data types
for i in ow.np_results.keys():
    print(i)

# Declare result node types from OutputWriter dict as set
nodetype = set()

for i in ow.np_results.keys():
    before = i.split('.')[0]
    nodetype.add(before)

No time steps to calculate are specified. I'll check the datasource of the first controller for avaiable time steps
c:\Users\Victus\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandapower\timeseries\output_writer.py:177: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'range(0, 96)' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  self.output["Parameters"].loc[:, "time_step"] = self.time_steps
c:\Users\Victus\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandapower\control\run_control.py:50: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  level = controller.level.fillna(0).apply(asarray).values
100%|██████████| 96/96 [00:16<00:00,  5.72it/s]

res_junction.p_bar
res_pipe.v_mean_m_per_s
res_pipe.reynolds
res_pipe.lambda
res_heat_consumer.mdot_from_kg_per_s
res_heat_consumer.t_from_k
res_heat_consumer.t_to_k
res_circ_pump_pressure.mdot_flow_kg_per_s
res_ext_grid.mdot_kg_per_s


In [16]:
## Declare datasets for visualization purposes

# Declare datasets
heat_consumer_data = pd.DataFrame(columns=('Massflow','T (flow side)','T (return side)'))

# Intermediate data formatting from pandapipes outputwriter
consumer_massflow = np.transpose(ow.np_results["res_heat_consumer.mdot_from_kg_per_s"])
consumer_t_from = np.transpose(ow.np_results["res_heat_consumer.t_from_k"])
consumer_t_to = np.transpose(ow.np_results["res_heat_consumer.t_to_k"])

# Input data to datasets
for i in range(0,len(net.heat_consumer.name)):
    heat_consumer_data.loc[i,'Massflow'] = list(consumer_massflow[i,:])
    heat_consumer_data.loc[i,'T (flow side)'] = list(consumer_t_from[i,:])
    heat_consumer_data.loc[i,'T (return side)'] = list(consumer_t_to[i,:])

heat_consumer_data['Name'] = net.heat_consumer.name
heat_consumer_data['Type'] = net.heat_consumer.type

# Declare consumer types for filtering
consumer_types = set()

for i in net.heat_consumer.type:
    consumer_types.add(i)

heat_consumer_data.head()

,Massflow,T (flow side),T (return side),Name,Type
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[358.15, 358.15, 358.15, 358.15, 358.15, 358.1...","[308.15, 308.15, 308.15, 308.15, 308.15, 308.1...",Sink_129107,Omakotitalot
1,"[0.268, 0.268, 0.268, 0.268, 0.266, 0.266, 0.2...","[358.15, 358.15, 358.15, 358.15, 358.15, 358.1...","[308.15, 308.15, 308.15, 308.15, 308.15, 308.1...",Sink_127293,Rivitalot
2,"[0.289, 0.289, 0.289, 0.289, 0.286, 0.286, 0.2...","[358.15, 358.15, 358.15, 358.15, 358.15, 358.1...","[308.15, 308.15, 308.15, 308.15, 308.15, 308.1...",Sink_127222,Asuinkerrostalot
3,"[0.207, 0.207, 0.207, 0.207, 0.206, 0.206, 0.2...","[358.15, 358.15, 358.15, 358.15, 358.15, 358.1...","[308.15, 308.15, 308.15, 308.15, 308.15, 308.1...",Sink_128285,Asuinkerrostalot
4,"[0.273, 0.273, 0.273, 0.273, 0.271, 0.271, 0.2...","[358.15, 358.15, 358.15, 358.15, 358.15, 358.1...","[308.15, 308.15, 308.15, 308.15, 308.15, 308.1...",Sink_128595,Asuinkerrostalot


In [17]:
## Grouped, interactive HoloViz visualisation
# Create tabbed interactive visualisations for different node and data types.
# TODO: tabs

# Explode data for heatmap so each massflow state corresponds to own row
cons_massflow_expanded = heat_consumer_data.explode('Massflow')

# Create an x-axis range for each value in Massflow
cons_massflow_expanded['x'] = list(range(96)) * len(heat_consumer_data['Name'])

# Reset index after explosion to clean up the DataFrame
cons_massflow_expanded = cons_massflow_expanded.reset_index(drop=True)

# Declare CrossSelector for consumer type filtering
type_selector = pn.widgets.CrossSelector(name='Filter by Consumer Type',
                                        value=heat_consumer_data['Type'].unique().tolist(), 
                                        options=heat_consumer_data['Type'].unique().tolist()
                                        )

# Declare default HeatMap to define initial state
heatmap_default = hv.HeatMap(consumer_massflow, label='Consumer Mass Flow').opts(
    title="Consumer Mass Flow Profiles",
    width=960, height=10*len(net.heat_consumer.name),
    xlabel="Time Step", ylabel="Heat Consumer",
    xlim=(-0.5,n_ts),
    ylim=(0,len(heat_consumer_data)),
    shared_axes=False,
    cmap="magma",
    toolbar='above',
    tools=['hover']+['crosshair'],
    backend_opts={"plot.toolbar.active_scroll": None,"plot.toolbar.active_drag":None}
    )

# Define a function to filter the DataFrame and create the heatmap
def create_heatmap(selected_types):
    
    # Filter the DataFrame based on the selected Type values
    filtered_df = cons_massflow_expanded[cons_massflow_expanded['Type'].isin(selected_types)]
    
    # Create a HoloViews dataset
    dataset = hv.Dataset(filtered_df, kdims=['x', 'Name'], vdims=['Massflow', 'Type'])

    # Calculate number of unique Name elements for Heatmap height
    num_names = filtered_df['Name'].nunique()

    # Calculate the minimum and maximum of the filtered Massflow data
    min_massflow = filtered_df['Massflow'].min()
    max_massflow = filtered_df['Massflow'].max()

    # Add 5% padding to the min and max values
    range_padding = (max_massflow - min_massflow) * 0.05
    min_massflow -= range_padding
    max_massflow += range_padding

    # Customize the hover tool to show the Type
    hover_tool = [
        ('Time Step', '@x'),
        ('Consumer', '@Name'),
        ('mdot kg / s', '@Massflow'),
        ('Type', '@Type')  # Add Type information to the hover tooltip
    ]

    # Declare HeatMap and specify options
    massflow_hm = hv.HeatMap(dataset, label='Consumer Mass Flow').opts(
        title="Consumer Mass Flow Profiles",
        width=960, height=10*num_names,
        xlabel="Time Step", ylabel="Heat Consumer",
        xlim=(-0.5,n_ts),
        ylim=(0,len(heat_consumer_data)),
        clim=(min_massflow, max_massflow),
        shared_axes=False,
        cmap="magma",
        toolbar='above',
        tools=['hover']+['crosshair'],
        hover_tooltips=hover_tool,
        backend_opts={"plot.toolbar.active_scroll": None,"plot.toolbar.active_drag":None}
        )
    
    return massflow_hm

# Create the dynamic heatmap using pn.depends on type_selector
@pn.depends(type_selector.param.value)
def dynamic_heatmap(selected_types):
    return create_heatmap(selected_types)

# Create a Tap stream for the heatmap to capture clicks
heatmap_tapsource = hv.DynamicMap(dynamic_heatmap)
tap_stream = hv.streams.Tap(source=heatmap_tapsource, x=None, y=None)


# Function to create a line plot when a Name is tapped on the heatmap
def create_line_plot(x, y):
    # If no name is selected, return default consumer
    if y is None:
        return hv.Curve(consumer_massflow[1,:]).opts(width=400, height=400, tools=['hover'])
    
    # Get the selected Name
    selected_name = y
    
    # Filter the DataFrame based on the selected Type values
    filtered_df = heat_consumer_data[heat_consumer_data['Name'] == selected_name]

    # Create a HoloViews dataset
    dataset_curve = hv.Dataset(cons_massflow_expanded, kdims=['Name'], vdims=['Massflow'])
    
    # If the name is not found, return default consumer
    if filtered_df.empty:
        return hv.Curve([]).opts(width=400, height=400, xlabel='Time step', ylabel='Massflow')
    
    # Get the massflow values and plot them against the time steps (0-96)
    massflow_values = filtered_df['Massflow']
    
    # Create a line plot
    line_plot = hv.Curve(dataset_curve.select(Name=y), kdims=['x'], vdims=['Massflow']).opts(
                         width=400, height=400, tools=['hover'], xlabel='Time step', ylabel='Massflow'
    )

    print(filtered_df['Massflow'].values[0])
    return line_plot

# Create a DynamicMap for the line plot that updates when the Tap stream changes
dynamic_line_plot = hv.DynamicMap(create_line_plot, streams=[tap_stream])

# Declare line graphs
massflow_curve = hv.Curve(consumer_massflow[1,:])
t_from_curve = hv.Curve(consumer_t_from[1,:])
t_to_curve = hv.Curve(consumer_t_to[1,:])

# Specify line graph options
massflow_curve.opts(title=str(net.heat_consumer.name[1])+" Mass Flow Profile",
                    xlabel="Time Step", ylabel="mdotkg / s",
                    xlim=(-0.5,n_ts),
                    ylim=(np.min(consumer_massflow[1,:])
                          - np.mean(consumer_massflow[1,:])/20,
                          np.max(consumer_massflow[1,:])
                          + np.mean(consumer_massflow[1,:])/20
                          ),
                    shared_axes=False,
                    tools=['hover'],
                    framewise=True,
                    backend_opts={"plot.toolbar.active_scroll": None,"plot.toolbar.active_drag":None}
                    )

t_from_curve.opts(title=str(net.heat_consumer.name[1])+" Flow Side Temp",
                  xlabel="Time Step", ylabel="K",
                  xlim=(-0.5,n_ts),
                  ylim=(np.min(consumer_t_from[1,:])
                        - np.mean(consumer_t_from[1,:])/20,
                        np.max(consumer_t_from[1,:])
                        + np.mean(consumer_t_from[1,:])/20
                        ),
                  shared_axes=False,
                  tools=['hover'],
                  framewise=True,
                  backend_opts={"plot.toolbar.active_scroll": None,"plot.toolbar.active_drag":None}
                  )

t_to_curve.opts(title=str(net.heat_consumer.name[1])+" Return Side Temp",
                xlabel="Time Step", ylabel="K",
                xlim=(-0.5,n_ts),ylim=(np.min(consumer_t_to[1,:])-np.mean(consumer_t_to[1,:])/20,np.max(consumer_t_to[1,:])+np.mean(consumer_t_to[1,:])/20),
                shared_axes=False,
                tools=['hover'],
                framewise=True,
                backend_opts={"plot.toolbar.active_scroll": None,"plot.toolbar.active_drag":None}
                )

# Define layout
right_column = pn.Column(dynamic_line_plot,t_from_curve,t_to_curve)

layout = pn.Column(pn.Row(pn.pane.Str(width=130, height=220),type_selector,max_height=300),
                   pn.Row(pn.panel(dynamic_heatmap),right_column))

layout.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'59f8bf21-b978-40b1-a24b-aa0aabeaa4b4': {'version…

In [18]:
hv.help(hv.HeatMap)

HeatMap

Online example: https://holoviews.org/reference/elements/bokeh/HeatMap.html

-------------
Style Options
-------------

	xmarks_visible, xmarks_muted, xmarks_line_color, xmarks_line_alpha, xmarks_color, xmarks_alpha, xmarks_line_width, xmarks_line_join, xmarks_line_cap, xmarks_line_dash, xmarks_line_dash_offset, xmarks_selection_line_color, xmarks_nonselection_line_color, xmarks_muted_line_color, xmarks_hover_line_color, xmarks_selection_line_alpha, xmarks_nonselection_line_alpha, xmarks_muted_line_alpha, xmarks_hover_line_alpha, xmarks_selection_color, xmarks_nonselection_color, xmarks_muted_color, xmarks_hover_color, xmarks_selection_alpha, xmarks_nonselection_alpha, xmarks_muted_alpha, xmarks_hover_alpha, xmarks_selection_line_width, xmarks_nonselection_line_width, xmarks_muted_line_width, xmarks_hover_line_width, xmarks_selection_line_join, xmarks_nonselection_line_join, xmarks_muted_line_join, xmarks_hover_line_join, xmarks_selection_line_cap, xmarks_nonselection_line_cap

In [19]:
pn.extension()

# Sample DataFrame setup
df = pd.DataFrame({
    'Name': ['A', 'B', 'C', 'D', 'E'],
    'Type': ['Type1', 'Type2', 'Type1', 'Type2', 'Type3'],
    'Massflow': [np.random.rand(96).tolist() for _ in range(5)]  # 96 random values per Name
})

# Creating the CrossSelector for filtering by Type
selector = pn.widgets.CrossSelector(name='Select Types', options=list(df['Type'].unique()))

# Function to create the heatmap based on selected types
def create_heatmap(selected_types):
    # Filter the dataframe based on the selected types
    filtered_df = df[df['Type'].isin(selected_types)]
    
    # If no selection, return empty heatmap
    if filtered_df.empty:
        return hv.HeatMap([]).opts(width=800, height=400, colorbar=True, cmap='Viridis')
    
    # Prepare data for the heatmap
    heatmap_data = []
    for idx, row in filtered_df.iterrows():
        for i, value in enumerate(row['Massflow']):
            heatmap_data.append((i, row['Name'], value))
    
    heatmap = hv.HeatMap(heatmap_data, kdims=['x', 'Name'], vdims='Massflow').opts(
        colorbar=True, width=800, height=400, cmap='Viridis', tools=['hover']
    )
    
    return heatmap

# Function to handle dynamic map callback
def dynamic_heatmap_callback(value):
    return create_heatmap(value)

# Bind the heatmap creation function to the selector's value
dynamic_heatmap = hv.DynamicMap(create_heatmap(selector.values), streams=[pn.bind(lambda: selector.value, selector)])

# Panel layout to display the selector and the heatmap
layout = pn.Column(selector, dynamic_heatmap)

# Display the layout in a notebook or server
layout.servable()

ValueError: Callable parameter 'Callable.callable' only takes a callable object, not objects of <class 'holoviews.element.raster.HeatMap'>.